In [1]:
import yfinance as yf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense


1. get data
2. prepare data
3. standardize data
4. train/test split
5. build training and predicting pipeline: that allows to quickly train and evaluate models

In [2]:
df=yf.download('DIS', start='2023-01-01', end='2023-07-30')

[*********************100%***********************]  1 of 1 completed


In [3]:
#set cutoff date for training and testing data
test_size = 30


In [5]:
columns_not_scale = ['Date', 'Close', 'Adj Close']
X_to_scale = df.drop(columns_not_scale, axis=1)

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [7]:
# Separate features and target, also take Date out as we are not goig to scale this data
columns_to_drop = ['Close', 'Adj Close']
X = df.drop(columns_to_drop, axis=1)
y = df['Adj Close']

In [18]:
X.head()

Open       High        Low    Volume
Date                                                 
2023-01-03  88.980003  89.970001  87.830002  14997100
2023-01-04  90.000000  92.750000  89.360001  14957200
2023-01-05  91.660004  92.480003  90.510002  11622600
2023-01-06  92.660004  94.690002  91.320000   9828100
2023-01-09  94.430000  95.699997  93.449997  11675800

In [10]:
# Standardize features using MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_to_scale)

In [ ]:
#reassemble the X dataframe to include date again
X_scaled_df = pd.DataFrame(X_scaled, columns=X)
X_scaled_df['Date'] = X['Date'].values

In [13]:
# Split scaled data into training and testing sets
X_train_scaled = X_scaled_df[:-test_size]
X_test_scaled = X_scaled_df[-test_size:]
y_train = y.iloc[:-test_size]
y_test = y.iloc[-test_size:]

In [14]:
X_train_scaled.shape

(113, 4)

In [16]:
X_test_scaled.shape

(30, 4)

In [17]:
y_train.shape

(113,)

In [ ]:
#reshape data for lstm: LSTM models expect input data in a 3D shape (samples, time steps, features)
#time steps: is the number of times periods in the dataset(days, months...)that you want to take as input for prediction
# Reshape X_train_scaled
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 5, X_train_scaled.shape[1]))

# Reshape X_test_scaled
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 5, X_test_scaled.shape[1]))


In [ ]:
#build and train the LSTM Model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, num_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train_reshaped, y_train, epochs=50, batch_size=16, verbose=1)


In [ ]:
#make predictions and evaluate model
y_pred = model.predict(X_test_reshaped)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
